# Sparse Subspace Clustering (Vidal IEEE Trans. on PAMI 2013) on Wine Dataset

#### References:


1.   Main Paper (Vidal IEEE Trans. on PAMI 2013): https://arxiv.org/pdf/1203.1005.pdf
2.   Supplementary Paper (Vidal CVPR 2009): http://cis.jhu.edu/~ehsan/Downloads/SSC-CVPR09-Ehsan.pdf
3.   Spectral Clustering: http://people.csail.mit.edu/dsontag/courses/ml14/notes/Luxburg07_tutorial_spectral_clustering.pdf
4.   Spectral Clustering Code: https://juanitorduz.github.io/spectral_clustering/
5.   Wine Dataset: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html

Our objective is given a set of points drawn from a `union of subspaces`, we need to find the number of subspaces, their dimensions, a basis for each subspace, and the segmentation of the data.

In [99]:
import numpy as np
import cvxpy as cp
from cvxpy.atoms.elementwise.power import power

##### Set Parameters of the data space

In [100]:
N = 178 # Total no of points
D = 13  # Dimension of space
K = 3 # Number of clusters

In [101]:
from sklearn.datasets import load_wine
wine_data = load_wine()
wine_X = np.asarray(wine_data.data)
input_data = wine_X.T

The matrix *input_data* contains the data points arranged sequentially forming a matrix of shape $(D,N)$. The *input_data* matrix is of the form $[y[0],y[1], \ldots, y[N-1]], $where each $y_i$ denotes the vector of a data point.

Also, *input_data* can be assumed to be of the form $[Y_0, Y_1, \ldots, Y_n]$, where each $Y_i$ denotes the set of $N_i$ data points belonging to subspace $i$. Also assume that the dimension of each subspace $i$ is $d_i$ and $A_i$ is its basis.

In [102]:
def find_sparse_sol(Y,i,N):
    if i is 0:
        Ybari = Y[:,1:N]
    if i is N-1:
        Ybari = Y[:,0:N-1]
    if i!=0 and i!=N-1:
        Ybari = np.concatenate((Y[:,0:i],Y[:,i+1:N]),axis=1)
    yi = Y[:,i].reshape(D,1)
    
    # this ci will contain the solution of the l1 optimisation problem:  
    # min (||yi - Ybari*ci||F)^2 + lambda*||ci||1   st. sum(ci) = 1
    
    ci = cp.Variable(shape=(N-1,1))
    constraint = [cp.sum(ci)==1]
    obj = cp.Minimize(power(cp.norm(yi-Ybari*ci,2),2) + 1.957*cp.norm(ci,1)) #lambda = 1.957
    prob = cp.Problem(obj, constraint)
    prob.solve()
    return ci.value

The above function `find_sparse_sol` finds the solution to the following optimisation problem $(\textrm{min} \; ||y_i - Y_{\hat{i}}c_i||_{F}^2 + \lambda||c_i||_1)$ subject to the constraint $\sum_{i=i}^{N}c_i=1$. We use this optimisation term to *deal with all the possible noise values in the data.*

In [103]:
C = np.concatenate((np.zeros((1,1)),find_sparse_sol(input_data,0,N)),axis=0)

for i in range(1,N):
    ci = find_sparse_sol(input_data,i,N)
    zero_element = np.zeros((1,1))
    cif = np.concatenate((ci[0:i,:],zero_element,ci[i:N,:]),axis=0)
    C = np.concatenate((C,cif),axis=1)
print(C.shape)

(178, 178)


We now include a zero vector of size $(1,1)$ at the $i$ th position of $c_i$ to form $\hat{c_i}$ which we represent as *cif* in the code.
Then we concatenate $[\hat{c_1}$, $\hat{c_2}$, ..., $\hat{c_N}]$ to form the matrix $C$.  

$C$ is the *Matrix of Coefficients* and is of the form $C = [\hat{c_1}, \hat{c_2}, ..., \hat{c_N}] \in \mathbb{R}^{NXN}$.

In [104]:
W = np.add(np.absolute(C), np.absolute(C.T))
print(W.shape)

(178, 178)


In [105]:
# Check sparsity by counting the number of zeros

cz = 0
for i in range(W.shape[0]):
    for j in range(W.shape[1]):
        if W[i,j] < 1e-5 and W[i,j] > -1e-5:
            cz += 1
print(cz)

28954


In the above code block, we make $C$ a symmetric matrix by the operation $W = |C| + |C^T|$. It is still a valid representation of the similarity since if $y_i$ can write itself as a linear combination of all the points in the same subspace including $y_j$, then $y_j$ can also be represented as a linear combination of all the other points belonging to the same subspace including $c_i$.

In [106]:
D = np.zeros((N,N))
sum_list=[]
for i in range(N):
    csum = np.sum(W[:,i],axis=0)
    sum_list.append(csum)

D = np.diag(sum_list)
print(D)

[[1.77566829 0.         0.         ... 0.         0.         0.        ]
 [0.         3.0934134  0.         ... 0.         0.         0.        ]
 [0.         0.         1.26613727 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 4.42537469 0.         0.        ]
 [0.         0.         0.         ... 0.         1.67255326 0.        ]
 [0.         0.         0.         ... 0.         0.         2.04561665]]


$D$ is the `degree matrix`. In this case, $D \in \mathbb{R}^{NxN}$ is a diagonal matrix with $D_{ii} = \sum_{j}W_{ij}$. 

In [107]:
L = np.subtract(D, W)
print(L.shape)

(178, 178)


This $L$ is the Laplacian matrix, which can be defined as $L = D - W$. Next, we calculate the `eigenvalues` and `eigenvectors` of the Laplacian matrix, which we will use for Spectral clustering of the data points. $L$ is a *positive, semi-definite matrix* this means all the eigenvalues of the matrix will be greater than equal to $0$.

### Perform Spectral Clustering with Laplacian Matrix L

In [108]:
from scipy import linalg

eigenvals, eigenvcts = linalg.eig(L)

eigenvals = np.real(eigenvals)
eigenvcts = np.real(eigenvcts)

eig = eigenvals.reshape((N,1))

Sort Eigen Values

In [109]:
eigenvals_sorted_indices = np.argsort(eigenvals)
eigenvals_sorted = eigenvals[eigenvals_sorted_indices]

In [110]:
indices = []
for i in range(0,K):
    ind = []
    print(eigenvals_sorted_indices[i])
    ind.append(eigenvals_sorted_indices[i])
    indices.append(np.asarray(ind))

3
52
62


In the above code, we find out the indices of the eigenvectors corresponding to the $K$ smallest eigenvectors.

In [111]:
indices = np.asarray(indices)

In [112]:
zero_eigenvals_index = np.array(indices)

Here, the indices are arranged according to their sorted order of values and the sorted eigen values are stored in *eigenvals_sorted*.

In [113]:
eigenvals[zero_eigenvals_index]

array([[4.18428276e-15],
       [4.07086925e-01],
       [4.68974664e-01]])

Here, we collect the indices of *the zero eigen values* (for convenience we consider values smaller than $10^{-5}$ to be equal to zero). \\
*The zero eigen values* are the displayed.

In [114]:
import pandas as pd

proj_df = pd.DataFrame(eigenvcts[:, zero_eigenvals_index.squeeze()])
proj_df.columns = ['v_' + str(c) for c in proj_df.columns]
proj_df.head()

,v_0,v_1,v_2
0,0.074953,-0.070832,-0.096227
1,0.074953,-0.056307,-0.047522
2,0.074953,-0.058465,-0.019998
3,0.074953,-0.062030,-0.080295
4,0.074953,-0.012187,-0.034480


Stack the Eigen Vectors corresponding to the zero Eigen Values in a dataframe *proj_df*. This can be thought of as a $N X Z$ matrix ($Z$ = number of zero eigen values) where the columns denote an eigen vector and the rows denote a data point.

Apply *K-Means Clustering* with $K = 3$.

In [115]:
from sklearn.cluster import KMeans

def run_k_means(df, n_clusters):
    k_means = KMeans(random_state=25, n_clusters=n_clusters)
    k_means.fit(df)
    cluster = k_means.predict(df)
    return cluster

cluster = run_k_means(proj_df, n_clusters=K)

*run_k_means* applies `K-Means Clustering` on *proj_df* with number of clusters = $2$. \\
The clustering of the data points is returned in the variable *'cluster'*.

Display clusters formed

In [116]:
print(cluster)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 2 2 2 0 2 0 2 2 2 2 0
 0 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 0 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2
 2 0 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


As we can see, the data points have been clustered into two subspaces: $0$ and $1$ corresponding to the $2$ subspaces that we have considered.

In [117]:
c0 = 0
for l in cluster:
    if l == 0:
        c0 += 1
print(c0)

68


In [118]:
c1 = 0
for l in cluster:
    if l == 1:
        c1 += 1
print(c1)

49


In [119]:
c2 = 0
for l in cluster:
    if l == 2:
        c2 += 1
print(c2)

61


So we find that $68$ data points have been labelled $0$, signifying that they belong to  the $1$st subspace, while $49$ have been labelled $1$ and the remaining $61$ points have been labelled $2$. 

In [120]:
orig = wine_data.target

In [121]:
pred = np.asarray(cluster)

### Calculate Performance

In [122]:
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import normalized_mutual_info_score
print("ARI = " + str(adjusted_rand_score(orig, pred)))
print("NMI = " + str(normalized_mutual_info_score(orig, pred)))

ARI = 0.8297415339237041
NMI = 0.8358558565678333


In the above code block, we calculate the `Adjusted random score` and the `normalized mutual info score` between the `original` and the `predicted` labels for the various data points.